In [1]:
from openai import OpenAI
from agents import Agent, InputGuardrail, GuardrailFunctionOutput, Runner
from pydantic import BaseModel
import asyncio
import os
import base64
from IPython.display import Image, display
import json

In [ ]:
# Load the OpenAI API key from the environment
openai_key = os.environ["OPENAI_API_KEY"]
client = OpenAI(api_key=openai_key)

In [5]:
# Simple Text Generation
response = client.responses.create(
    model="gpt-4o",
    input="tell me a joke",
)

print(response.output_text)

Why don't skeletons fight each other?

They don't have the guts.


In [9]:
# Multi-Modal Generation Text and Image 

image_Url = 'https://t4.ftcdn.net/jpg/01/71/94/77/360_F_171947753_jI2NCXjEGNscOaYhV0wvPv4vFSDnnwUM.jpg'

response = client.responses.create(
    model = "gpt-4o",
    input = [
        {"role": "user",
         "content": "what teams are playing in this image?"
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "input_image",
                    "image_url": image_Url
                }
            ]
        }
    ]
)

print(response.output_text)

The image shows children playing soccer in a park. They don't appear to be part of any specific teams; it looks like a casual game.
